In [1]:
!pip install yfinance

In [2]:
# For data manipulation
import numpy as np
import pandas as pd

# To fetch financial data
import yfinance as yf

In [3]:
# Set the ticker as 'EURUSD=X'
forex_data = yf.download('EURJPY=X',
                        # start='2019-01-02',
                        #  end='2021-12-31'
                         )

# Set the index to a datetime object
forex_data.index = pd.to_datetime(forex_data.index)

# Display the last five rows
forex_data.head()

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,EURJPY=X,EURJPY=X,EURJPY=X,EURJPY=X,EURJPY=X
Date,,,,,
2003-01-23,126.879997,127.110001,126.519997,126.980003,0
2003-01-24,127.489998,127.760002,126.800003,126.900002,0
2003-01-27,128.440002,128.940002,127.489998,127.589996,0
2003-01-28,128.470001,128.720001,128.020004,128.460007,0
2003-01-29,128.279999,128.619995,127.889999,128.460007,0


In [4]:
# Change forex_data columns from multiindex to regular
forex_data.columns = [col[0] for col in forex_data.columns]

forex_data.drop(columns=['Volume'], inplace=True)



In [5]:
# Feature calculation functions

# Calculate Moving Average (MA)
def calculate_ma(data, period):
    return data['Close'].rolling(window=period).mean()

# Calculate Bollinger Bands (BB)
def calculate_bb(data, period, num_std):
    rolling_mean = data['Close'].rolling(window=period).mean()
    rolling_std = data['Close'].rolling(window=period).std()
    upper_band = rolling_mean + (rolling_std * num_std)
    lower_band = rolling_mean - (rolling_std * num_std)
    return upper_band, lower_band

# Calculate Average True Range (ATR)
def calculate_atr(data, period):
    high_low = data['High'] - data['Low']
    high_close_prev = abs(data['High'] - data['Close'].shift(1))
    low_close_prev = abs(data['Low'] - data['Close'].shift(1))
    tr = pd.concat([high_low, high_close_prev, low_close_prev], axis=1).max(axis=1)
    return tr.rolling(window=period).mean()

# Calculate Relative Strength Index (RSI)
def calculate_rsi(data, period):
    delta = data['Close'].diff(1)
    up = delta.clip(lower=0)
    down = -1 * delta.clip(upper=0)
    ma_up = up.rolling(window=period).mean()
    ma_down = down.rolling(window=period).mean()
    rsi = 100 - (100 / (1 + ma_up / ma_down))
    return rsi

# Calculate Stochastic Oscillator (%K and %D)
def calculate_stochastic(data, period_k, period_d):
    low_min = data['Low'].rolling(window=period_k).min()
    high_max = data['High'].rolling(window=period_k).max()
    k = 100 * (data['Close'] - low_min) / (high_max - low_min)
    d = k.rolling(window=period_d).mean()
    return k, d

def calculate_ema(data, period):
    return data['Close'].ewm(span=period, adjust=False).mean()


# Calculate MACD (Moving Average Convergence Divergence)
def calculate_macd(data, short_period, long_period, signal_period):
    short_ema = calculate_ema(data, short_period)
    long_ema = calculate_ema(data, long_period)
    macd = short_ema - long_ema
    signal = macd.ewm(span=signal_period, adjust=False).mean()
    histogram = macd - signal
    return macd, signal, histogram

In [6]:
# Add feature columns

forex_data['MA_20'] = calculate_ma(forex_data, 20)
forex_data['MA_50'] = calculate_ma(forex_data, 50)

forex_data['BB_Upper_20'], forex_data['BB_Lower_20'] = calculate_bb(forex_data, 20, 2)

forex_data['ATR_14'] = calculate_atr(forex_data, 14)

forex_data['RSI_14'] = calculate_rsi(forex_data, 14)

forex_data['Stochastic_K_14'], forex_data['Stochastic_D_3'] = calculate_stochastic(forex_data, 14, 3)

forex_data['EMA_20'] = calculate_ema(forex_data, 20)

forex_data['MACD'], forex_data['MACD_Signal'], forex_data['MACD_Histogram'] = calculate_macd(forex_data, 12, 26, 9)


In [7]:
## IMPORTANT STEP
# Create Target variable based on Next 7 Days Average Close

forex_data['Avg_Close_Next_7_Days'] = forex_data['Close'].rolling(window=7, min_periods=1).mean().shift(-7)

forex_data['Target'] = np.where(forex_data['Avg_Close_Next_7_Days'] > forex_data['Close'], 1, 0)

# Display the last 15 rows
forex_data.tail(15)

,Close,High,Low,Open,MA_20,MA_50,BB_Upper_20,BB_Lower_20,ATR_14,RSI_14,Stochastic_K_14,Stochastic_D_3,EMA_20,MACD,MACD_Signal,MACD_Histogram,Avg_Close_Next_7_Days,Target
Date,,,,,,,,,,,,,,,,,,
2025-02-28,155.927994,157.139008,154.813004,155.927994,158.009749,160.721705,161.054245,154.965253,1.766499,46.555583,17.523051,13.933176,158.088931,-1.212841,-1.052244,-0.160596,158.979283,1
2025-03-03,156.996994,158.451996,155.927002,156.996994,157.899748,160.593805,160.922526,154.876971,1.837498,53.306346,34.323329,19.301087,157.984937,-1.149713,-1.071738,-0.077975,159.631712,1
2025-03-04,156.533997,157.145996,155.643005,156.533997,157.708898,160.458405,160.556974,154.860822,1.634283,39.772675,27.046918,26.297766,157.846752,-1.124087,-1.082208,-0.041879,160.327855,1
2025-03-05,159.141006,160.468994,158.776001,159.141006,157.672798,160.372505,160.422516,154.923081,1.709783,43.794036,76.520688,45.963645,157.970015,-0.883232,-1.042413,0.159181,160.535998,1
2025-03-06,160.720993,161.253998,159.134995,160.720993,157.773198,160.319525,160.814055,154.732340,1.800212,53.230530,91.724806,65.097471,158.232013,-0.558425,-0.945615,0.387191,160.672998,0
2025-03-07,159.759003,160.712997,159.292999,159.759003,157.910298,160.246545,161.053144,154.767453,1.795212,51.392859,76.789374,81.678289,158.377440,-0.374322,-0.831356,0.457035,161.149569,1
2025-03-10,160.302994,160.276993,158.953995,160.302994,158.096998,160.163505,161.346186,154.847810,1.812283,57.043759,85.235136,84.583105,158.560826,-0.182420,-0.701569,0.519149,161.602855,1
2025-03-11,159.399994,161.703003,158.988998,159.399994,158.250648,160.058025,161.437447,155.063848,1.906712,52.842964,66.574612,76.199707,158.640747,-0.102026,-0.581661,0.479635,161.891571,1
2025-03-12,161.563995,162.348999,161.223999,161.563995,158.413947,160.021584,161.928795,154.899100,1.945499,65.697200,89.583278,80.464342,158.919152,0.134751,-0.438378,0.573129,NaN,0


In [8]:
# Get last 5 days
forex_data_latest = forex_data.tail(15)

forex_data = forex_data.iloc[:-15]

In [9]:
# Drop NaN values resulting from calculations
forex_data = forex_data.dropna()

# Select features and target
features = ['Close','Open','High','Low','EMA_20', 'MACD', 'MA_20', 'MA_50', 'BB_Upper_20', 'BB_Lower_20', 'ATR_14', 'RSI_14', 'Stochastic_K_14', 'Stochastic_D_3']
X = forex_data[features]
y = forex_data['Target']

In [10]:
# from sklearn.preprocessing import MinMaxScaler

# # Create a MinMaxScaler object
# scaler = MinMaxScaler()

# # Fit the scaler to the features and transform them
# X_normalized = scaler.fit_transform(X)

# # Convert the normalized features back to a DataFrame (optional)
# X_normalized_df = pd.DataFrame(X_normalized, columns=features)

In [11]:
from sklearn.preprocessing import StandardScaler

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Reshape for Conv1D and LSTM
X_reshaped = X_scaled.reshape(X_scaled.shape[0], X_scaled.shape[1], 1)

In [12]:
from sklearn.model_selection import train_test_split

# Randomize the data and split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_reshaped, y, test_size=0.2, random_state=42)

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, LSTM, Dense, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report

In [14]:
model1 = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)),
    Conv1D(filters=32, kernel_size=3, activation='relu'),
    LSTM(50, activation='relu'),
    Dense(1, activation='sigmoid')
])

model1.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
model1.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)
y_pred1 = (model1.predict(X_test) > 0.50).astype(int)
print("Model 1 Report:")
print(classification_report(y_test, y_pred1))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
114/114 ━━━━━━━━━━━━━━━━━━━━ 12s 17ms/step - accuracy: 0.5294 - loss: 0.6917 - val_accuracy: 0.5192 - val_loss: 0.6911
Epoch 2/50
114/114 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.5233 - loss: 0.6920 - val_accuracy: 0.5192 - val_loss: 0.6912
Epoch 3/50
114/114 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.5229 - loss: 0.6915 - val_accuracy: 0.5203 - val_loss: 0.6904
Epoch 4/50
114/114 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 0.5411 - loss: 0.6889 - val_accuracy: 0.5236 - val_loss: 0.6897
Epoch 5/50
114/114 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.5404 - loss: 0.6885 - val_accuracy: 0.5203 - val_loss: 0.6895
Epoch 6/50
114/114 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 0.5388 - loss: 0.6873 - val_accuracy: 0.5313 - val_loss: 0.6893
Epoch 7/50
114/114 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 0.5441 - loss: 0.6883 - val_accuracy: 0.5236 - val_loss: 0.6879
Epoch 8/50
114/114 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.5510 - loss: 0.6856 - val_acc

In [15]:
forex_data_latest_x = forex_data_latest[features]
forex_data_latest_y = forex_data_latest['Target']

latest_scaled = scaler.fit_transform(forex_data_latest)

# Reshape for Conv1D and LSTM
latest_reshaped = latest_scaled.reshape(latest_scaled.shape[0], latest_scaled.shape[1], 1)

pred = (model1.predict(latest_reshaped)>0.5).astype(int)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 259ms/step


In [16]:
test_data = forex_data_latest
test_data['Predicted'] = pred
test_data.tail(15)

,Close,High,Low,Open,MA_20,MA_50,BB_Upper_20,BB_Lower_20,ATR_14,RSI_14,Stochastic_K_14,Stochastic_D_3,EMA_20,MACD,MACD_Signal,MACD_Histogram,Avg_Close_Next_7_Days,Target,Predicted
Date,,,,,,,,,,,,,,,,,,,
2025-02-28,155.927994,157.139008,154.813004,155.927994,158.009749,160.721705,161.054245,154.965253,1.766499,46.555583,17.523051,13.933176,158.088931,-1.212841,-1.052244,-0.160596,158.979283,1,1
2025-03-03,156.996994,158.451996,155.927002,156.996994,157.899748,160.593805,160.922526,154.876971,1.837498,53.306346,34.323329,19.301087,157.984937,-1.149713,-1.071738,-0.077975,159.631712,1,0
2025-03-04,156.533997,157.145996,155.643005,156.533997,157.708898,160.458405,160.556974,154.860822,1.634283,39.772675,27.046918,26.297766,157.846752,-1.124087,-1.082208,-0.041879,160.327855,1,1
2025-03-05,159.141006,160.468994,158.776001,159.141006,157.672798,160.372505,160.422516,154.923081,1.709783,43.794036,76.520688,45.963645,157.970015,-0.883232,-1.042413,0.159181,160.535998,1,1
2025-03-06,160.720993,161.253998,159.134995,160.720993,157.773198,160.319525,160.814055,154.732340,1.800212,53.230530,91.724806,65.097471,158.232013,-0.558425,-0.945615,0.387191,160.672998,0,1
2025-03-07,159.759003,160.712997,159.292999,159.759003,157.910298,160.246545,161.053144,154.767453,1.795212,51.392859,76.789374,81.678289,158.377440,-0.374322,-0.831356,0.457035,161.149569,1,1
2025-03-10,160.302994,160.276993,158.953995,160.302994,158.096998,160.163505,161.346186,154.847810,1.812283,57.043759,85.235136,84.583105,158.560826,-0.182420,-0.701569,0.519149,161.602855,1,0
2025-03-11,159.399994,161.703003,158.988998,159.399994,158.250648,160.058025,161.437447,155.063848,1.906712,52.842964,66.574612,76.199707,158.640747,-0.102026,-0.581661,0.479635,161.891571,1,0
2025-03-12,161.563995,162.348999,161.223999,161.563995,158.413947,160.021584,161.928795,154.899100,1.945499,65.697200,89.583278,80.464342,158.919152,0.134751,-0.438378,0.573129,NaN,0,0
